In [1]:
#Imports
%matplotlib inline
from scipy import stats

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#list of players stats datafromes
lst_players_df = []
#over range 0-16 read csv into list
for i in range(17):
    if i < 10:
        lst_players_df.append(pd.read_csv('clean_players_200'+str(i)+'.csv'))
    else:
        lst_players_df.append(pd.read_csv('clean_players_20'+str(i)+'.csv'))
        
print(type(lst_players_df[0]))

<class 'pandas.core.frame.DataFrame'>


In [3]:
print(lst_players_df[15]['PTS'])

Series([], Name: PTS, dtype: object)


In [4]:
#list of average points/pergame for each season
lst_yearly_pts_avg = []
#list of points/pergame standard deviation for each season
lst_yearly_pts_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_pts_avg.append(np.mean(df['PTS']))#calculate avg points and append into list of avgs
    lst_yearly_pts_stddev.append(np.std(df['PTS']))#calculate stddev of points and append into list of stddevs


#list of average assist/pergame for each season
lst_yearly_ast_avg = [] 
#list of assist/pergame standard deviation for each season
lst_yearly_ast_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_ast_avg.append(np.mean(df['AST']))#calculate avg assist and append into list of avgs
    lst_yearly_ast_stddev.append(np.std(df['AST']))#calculate stddev of assist and append into list of stddevs

#list of average turnover/pergame for each season
lst_yearly_tov_avg = [] 
#list of turnover/pergame standard deviation for each season
lst_yearly_tov_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_tov_avg.append(np.mean(df['TOV']))#calculate avg turnover and append into list of avgs
    lst_yearly_tov_stddev.append(np.std(df['TOV']))#calculate stddev of turnover and append into list of stddevs

#list of average rebound/pergame for each season
lst_yearly_reb_avg = [] 
#list of rebound/pergame standard deviation for each season
lst_yearly_reb_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_reb_avg.append(np.mean(df['REB']))#calculate avg rebound and append into list of avgs
    lst_yearly_reb_stddev.append(np.std(df['REB']))#calculate stddev of rebound and append into list of stddevs

#list of average steal/pergame for each season
lst_yearly_stl_avg = [] 
#list of steal/pergame standard deviation for each season
lst_yearly_stl_stddev = []
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_stl_avg.append(np.mean(df['STL']))#calculate avg steal and append into list of avgs
    lst_yearly_stl_stddev.append(np.std(df['STL']))#calculate stddev of steal and append into list of stddevs

#list of average block/pergame for each season
lst_yearly_blk_avg = [] 
#list of block/pergame standard deviation for each season
lst_yearly_blk_stddev = [] 
for df in lst_players_df: #for each df in list of player stat df
    lst_yearly_blk_avg.append(np.mean(df['BLK']))#calculate avg block and append into list of avgs
    lst_yearly_blk_stddev.append(np.std(df['BLK']))#calculate stddev of block and append into list of stddevs
    
print(lst_yearly_pts_avg)
print(lst_yearly_pts_stddev)
print(lst_yearly_ast_avg)

[7.811337868480729, 7.982727272727271, 7.84929906542056, 7.745475113122178, 8.088146551724138, 7.982532751091705, 8.208733624454151, 8.267184035476715, 8.503820224719105, 8.571719457013574, 8.206194690265484, 7.928870292887029, 7.957995735607683, 8.090248962655599, 8.122560975609762, nan, 8.426748971193412]
[6.008464043349294, 5.895593255535084, 6.05682394750918, 5.811409426756881, 6.05255090245414, 6.293553966779834, 6.282485021625439, 6.108297276493315, 6.16054198433178, 5.8881114391191405, 5.776076433959757, 5.471363561842137, 5.490566151611964, 5.850609815030152, 5.494006263512788, nan, 6.051713402816103]
[1.8235827664399094, 1.841136363636363, 1.7829439252336448, 1.7585972850678726, 1.7741379310344811, 1.6890829694323146, 1.766157205240173, 1.8188470066518854, 1.7719101123595518, 1.8049773755656113, 1.7676991150442465, 1.746234309623431, 1.8085287846481861, 1.7804979253112028, 1.81951219512195, nan, 1.8302469135802475]


In [ ]:
#setting arbitraty weights
f_pts_weight = 1.0 #1 point is worth 1 points
f_reb_weight = 1.2 #rebounds don't necessarily lead to a score but give more oppotunities for them so estimated by 2.3 * .55(hish est fg%)
f_blk_weight = 2.3 #denial of a shot 2-3, twos are more common than threes so that is why it is 2.3 instead of 2.5
f_stl_wegiht = 1.8 #worth more than rebound due to likleyhood of fastbreak
f_ast_wegiht = 2.3 #always leads to a score so opposite of block
f_tov_weight = -1.8 #basically opposite of steal

In [ ]:
#calculating impact
index = 0
for df in lst_players_df:
    for i, row in df.iterrows():
        pts_zval = (row['PTS']-lst_yearly_pts_avg[index])/lst_yearly_pts_stddev[index]
        ast_zval = (row['AST']-lst_yearly_ast_avg[index])/lst_yearly_ast_stddev[index]
        reb_zval = (row['REB']-lst_yearly_reb_avg[index])/lst_yearly_reb_stddev[index]
        blk_zval = (row['BLK']-lst_yearly_blk_avg[index])/lst_yearly_blk_stddev[index]
        stl_zval = (row['STL']-lst_yearly_stl_avg[index])/lst_yearly_stl_stddev[index]
        tov_zval = (row['TOV']-lst_yearly_tov_avg[index])/lst_yearly_tov_stddev[index]
        row['IMPACT'] = (row['PTS']*f_pts_weight*math.e**(pts_zval/5) +\
                         (row['AST']*f_ast_weight*math.e**(ast_zval/5) +\
                          (row['REB']*f_reb_weight*math.e**(reb_zval/5) +\
                           (row['BLK']*f_blk_weight*math.e**(blk_zval/5) +\
                            (row['STL']*f_stl_weight*math.e**(stl_zval/5) +\
                             (row['TOV']*f_tov_weight*math.e**(tov_zval/5)
    index += 1